<center><h1> Modelo de machine learning para separação de fotos<br>
    em categorias de gato ou cachorro </h1></center>

### Instalando bibliotecas

In [ ]:
"""!pip install keras
!pip install tensorflow"""

### importando bibliotecas

In [2]:
%matplotlib inline

import os


import random
import numpy as np
import keras
import tensorflow as tf
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.utils import load_img, img_to_array 
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

Setando diretório raiz e porcentagens de treinamento e validação

In [5]:
root = 'PetImages'
train_split, val_split = 0.7, 0.15

categories = [x[0] for x in os.walk(root) if x[0]][1:]

print(categories)

['PetImages\\Cat', 'PetImages\\Dog']


Função para pré-processamento das imagens

In [6]:
def get_image(path):
    img = load_img(path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

Removendo arquivos corrompidos

In [7]:
removidos = []

for arquivo in os.listdir(f'{root}/Dog'):
  tamanho = os.path.getsize(f'{root}/Dog/{arquivo}')
  if tamanho == 0:
    os.remove(f'{root}/Dog/{arquivo}')
    removidos.append(f'Dog/{arquivo}')
  else:
    pass
for arquivo in os.listdir(f'{root}/Cat'):
  tamanho = os.path.getsize(f'{root}/Cat/{arquivo}')
  if tamanho == 0:
    os.remove(f'{root}/Cat/{arquivo}')
    removidos.append(f'Cat/{arquivo}')
  else:
    pass

print(removidos)

['Dog/11702.jpg', 'Cat/666.jpg']


Carregando as imagens

In [ ]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames 
              in os.walk(category) for f in filenames 
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

# contando o numero de classes
num_classes = len(categories)

In [10]:
print(data[0]

{'x': array([[[ -16.939003 ,   47.221    ,   79.32     ],
        [ -14.939003 ,   49.221    ,   81.32     ],
        [ -11.939003 ,   52.221    ,   84.32     ],
        ...,
        [  17.060997 ,   86.221    ,  120.32     ],
        [  19.060997 ,   85.221    ,  117.32     ],
        [  17.060997 ,   83.221    ,  115.32     ]],

       [[ -16.939003 ,   47.221    ,   79.32     ],
        [ -14.939003 ,   49.221    ,   81.32     ],
        [ -11.939003 ,   52.221    ,   84.32     ],
        ...,
        [  18.060997 ,   87.221    ,  121.32     ],
        [  19.060997 ,   85.221    ,  117.32     ],
        [  18.060997 ,   84.221    ,  116.32     ]],

       [[ -16.939003 ,   47.221    ,   79.32     ],
        [ -14.939003 ,   49.221    ,   81.32     ],
        [ -11.939003 ,   52.221    ,   84.32     ],
        ...,
        [  18.060997 ,   88.221    ,  119.32     ],
        [  20.060997 ,   86.221    ,  118.32     ],
        [  18.060997 ,   84.221    ,  116.32     ]],

       ...,



Randomizando dados

In [12]:
random.shuffle(data)

Separando dados de treinamento, validação e teste (70%, 15%, 15%)

In [ ]:
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]
print(data)

Separando dados por rótulos

In [ ]:
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]
print(y_test)

### Pré-processando os dados, garantindo float32 e normalizando em 0 e 1
Caso acuse erro de memória pule essa etapa

In [ ]:
# normalizando
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)

Resumo

In [ ]:
# summary
print("Carregadas %d imagens de %d categorias"%(len(data), num_classes))
print("treino / validação / teste: %d, %d, %d"%(len(x_train), len(x_val), len(x_test)))
print("shape dos dados treinados: ", x_train.shape)
print("shape dos rótulos treinados: ", y_train.shape)


Plotando imagens de amostra

In [ ]:
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(root) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
idx = [int(len(images) * random.random()) for i in range(8)]
imgs = [image.load_img(images[i], target_size=(224, 224)) for i in idx]
concat_image = np.concatenate([np.asarray(img) for img in imgs], axis=1)
plt.figure(figsize=(16,4))
plt.imshow(concat_image)

### Criando redes neurais


In [ ]:
# build the network
model = Sequential()
print("Input dimensions: ",x_train.shape[1:])

model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

In [ ]:
# compile the model to use categorical cross-entropy loss function and adadelta optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    validation_data=(x_val, y_val))


Plotando validação de loss e acurácia

In [ ]:
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_acc"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show()

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

## Transfer learning com rede já existente



In [ ]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

In [ ]:
# make a reference to VGG's input layer
inp = vgg.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(num_classes, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model_new = Model(inp, out)


In [ ]:
# make all layers untrainable by freezing weights (except for last layer)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

In [ ]:
history2 = model_new.fit(x_train, y_train, 
                         batch_size=128, 
                         epochs=10, 
                         validation_data=(x_val, y_val))


In [ ]:
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.plot(history2.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_acc"])
ax2.plot(history2.history["val_acc"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show()

In [ ]:
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

Predizendo novas imagens

In [ ]:
img, x = get_image('101_ObjectCategories/airplanes/image_0003.jpg')
probabilities = model_new.predict([x])
